## Лекция 2  BM5    

### TfidfVectorizer

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# инициализируем
vectorizer = TfidfVectorizer()

# составляем корпус документов
corpus = [
  'слово1 слово2 слово3',
  'слово2 слово3',
  'слово1 слово2 слово1',
  'слово4'
]

# считаем
X = vectorizer.fit_transform(corpus)
 
# получится следующая структура:
#        |  слово1  |  слово2  |  слово3  |  слово4
# текст1 |   0.6    |    0.5   |   0.6    |    0
# текст2 |   0      |    0.6   |   0.8    |    0
# текст3 |   0.9    |    0.4   |   0      |    0
# текст4 |   0      |    0     |   0      |    1
 
# чтобы получить сгенерированный словарь, из приведенной структуры TfidfVectorizer
# порядок совпадает с матрицей
vectorizer.get_feature_names()  # ['слово1', 'слово2', 'слово3', 'слово4']
 
# чтобы узнать индекс токена в словаре
vectorizer.vocabulary_.get('слово3') # вернет 2
 
# показать матрицу
X.toarray()
 
# теперь можно быстро подсчитать вектор для нового документа
new_doc = vectorizer.transform(['слово1 слово4 слово4']).toarray()  # результат [[0.36673901, 0, 0, 0.93032387]]
 

## Функция ранжирования bm25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [2]:
### реализуйте эту функцию ранжирования 
import collections
import string
import math
import pandas as pd
import numpy as np

from pymorphy2 import MorphAnalyzer
from razdel import tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

morph = MorphAnalyzer()
stop = set(stopwords.words('russian'))
k = 2.0
b = 0.75


def my_preprocess(text: str):
    text = str(text)
    text = text.replace("\n", " ").replace('/', ' ')
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokenized_text = list(tokenize(text))
    lemm = [morph.parse(i.text)[0].normal_form for i in tokenized_text]
    words = [i for i in lemm if i not in stop]
    return " ".join(words)


### __Задача 1__:    
Реализуйте поиск с метрикой *TF-IDF* через умножение матрицы на вектор.
Что должно быть в реализации:
- проиндексированная база, где каждый документ представлен в виде вектора TF-IDF
- функция перевода входяшего запроса в вектор по метрике TF-IDF
- ранжирование докуменов по близости к запросу по убыванию

В качестве корпуса возьмите корпус вопросов в РПН по Covid2019. Он состоит из:
> файл **answers_base.xlsx** - база ответов, у каждого ответа есть его номер, тематика и примеры вопросов, которые могут быть заданы к этому ответу. Сейчас проиндексировать надо именно примеры вопросов в качестве документов базы. Понимаете почему?

> файл **queries_base.xlsx** - вопросы юзеров, к каждому из которых проставлен номер верного ответа из базы. Разделите эти вопросы в пропорции 70/30 на обучающую (проиндексированную как база) и тестовую (как запросы) выборки. 


##### Функции для векторизации единичного запроса

In [3]:
def vec_tfidf(doc, vectorizer):
    doc = my_preprocess(doc)
    return vectorizer.transform([doc]).toarray()

##### Считаем данные, переименуем столбцы (для удобства), добавим 70% queries_base (путём разбиения на test/train - 0.3/0.7) и получим матрицу эмбеддингов

In [4]:
answers_df = pd.read_excel("answers_base.xlsx")
questions_df = pd.read_excel("queries_base.xlsx")

In [5]:
full_corpus = []
vectorizer = TfidfVectorizer()

answers_df.rename(columns={'Текст вопросов': 'text', 'Номер связки': 'join_num'}, inplace=True)
questions_df.rename(columns={'Текст вопроса': 'text', 'Номер связки\n': 'join_num'}, inplace=True)

train, test = train_test_split(questions_df, test_size=0.3)

answers_quest = pd.concat([answers_df, train])

for quest in answers_quest['text']:
    full_corpus.append(my_preprocess(quest))
    
X_train = vectorizer.fit_transform(full_corpus)
X_train.shape

(1652, 7417)

##### Возьмём случайный запрос из тестовой выборки

In [6]:
query = vec_tfidf(test.text.values[9], vectorizer)
print(test.text.values[9])


Где в Ростове на Дону мы можем сделать обязательный тест на коронавирус по медицскомк полису без оплаты
--
Отправлено из Mail.ru для Android четверг, 03 сентября 2020г., 13:12 +03:00 от covid@78cge.ru :

>Добрый день, Ваше сообщение зарегистрировано в системе под номером: 10853
>Ваш вопрос:
>У вас сказано, что добровольное взятие анализа платное, и взятие анализа без медицинских показаний тоже платное. А если у нас нет показаний и нас обязывают пройти этот тест - это же уже не добровольное прохождение, подскажите пожалуйста, где в Ростове на Дону мы сможем пройти обязательное тестирование по полису? Жду ответ -- Отправлено из Mail.ru для Android четверг, 03 сентября 2020г., 12:54 +03:00 от covid@78cge.ru :> Добрый день, Ваше сообщение зарегистрировано в системе под номером: 10844> Ваш вопрос:> Здравствуйте, мы 6 дней были в отпуске, а по приезду узнали об обязательном прохождении теста на коронавирус. Мы хотим его пройти, но у нас нет средств, а в поликлинике по страховому полису нам 

##### Путём перемножения матрицы на вектор получим рейтинг вопросов, соответствующих запросу

In [7]:
rating = X_train.dot(query.T)
rating.T[0]

array([0.0619214 , 0.03147497, 0.01378502, ..., 0.13371664, 0.04082632,
       0.05011642])

##### Выведем top 20 самых близких вопросов и посмотрим на их номер связки

In [8]:
buf = list(zip(rating.T[0], answers_quest['join_num'].values))

for counter, val in enumerate(sorted(buf, key=lambda x: -x[0])):
    print(str(val[0]) + ": " + str(int(val[1])))
    if counter >= 20:
        break

0.872465579747946: 6
0.8565766323060884: 37
0.8520109911095695: 308
0.8506964424341005: 308
0.5799577747276881: 308
0.5492405227665993: 308
0.46438488631836855: 6
0.4150960694233565: 308
0.4100833460262565: 57
0.38528174371736373: 308
0.3798309943298101: 308
0.3782031505152206: 37
0.3603318657711875: 308
0.3544300275326107: 6
0.34294649452108006: 6
0.33031712437246974: 5
0.29417124345478357: 308
0.29249321024959074: 308
0.2778121046352094: 6
0.2774299527163355: 308
0.2718401212379425: 308


### __Задача 2__:    
Аналогичная задаче1 с другой метрикой 

Реализуйте поиск с метрикой *BM25* через умножение матрицы на вектор. Что должно быть в реализации:

- проиндексированная база, где каждый документ представлен в виде вектора BM25
- функция перевода входяшего запроса в вектор по метрике BM25
- ранжирование докуменов по близости к запросу по убыванию

##### Функции для векторизации единичного запроса

In [9]:
def vec_bm25(doc, dict_words):
    vec = np.zeros((1, len(dict_words)))
    doc = my_preprocess(doc)
    for word in doc.split(" "):
        if word in dict_words.keys():
            vec[0, dict_words[word][-1]] = 1
    return vec

##### Обратно-индексированный словарь, по которому считаем метрики

In [10]:
def get_inverse_dict(mat):
    d = dict()
    mat = mat.toarray()
    for ind, word in enumerate(vectorizer.get_feature_names()):
        d[word] = [int(sum(mat[:, ind]))]
        for ind_j, doc_ind in enumerate(mat[:, ind].tolist()):
            if doc_ind != 0:
                d[word].append(doc_ind)
        d[word].append(ind)
    return d

##### Функции для векторизации "тренировочного" множества

In [11]:
def bm25_vectorizer(tf_val, len_d, corpus_len, nq):
    IDF = np.log((corpus_len-nq+0.5) / (nq+0.5))
    TF = (tf_val * (k+1)) / (tf_val + k * (1-b+b*(len_d / avrdl)))
    return TF * IDF

##### Создадим матрицу эмбеддингов

In [12]:
corpus_len = len(full_corpus)
avrdl = sum([len(i.split(" ")) for i in full_corpus]) / corpus_len

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(full_corpus)
inverse_dict = get_inverse_dict(X_train)

mat = np.zeros((corpus_len, len(inverse_dict)))

for ind, doc in enumerate(full_corpus):
    tokens = doc.split(" ")
    tf_values = collections.Counter(tokens)
    len_d = len(tokens)
    for word in tokens:
        if word not in inverse_dict.keys():
            continue
        mat[ind, inverse_dict[word][-1]] = bm25_vectorizer(tf_values[word],
                                                         len_d,
                                                         corpus_len,
                                                         len(inverse_dict[word]) - 1)


##### Также возьмём случайный запрос из тестовой выборки и посмотрим на рейтинг связок

In [13]:
query = vec_bm25(test.text.values[9], inverse_dict)
print(test.text.values[9])


Где в Ростове на Дону мы можем сделать обязательный тест на коронавирус по медицскомк полису без оплаты
--
Отправлено из Mail.ru для Android четверг, 03 сентября 2020г., 13:12 +03:00 от covid@78cge.ru :

>Добрый день, Ваше сообщение зарегистрировано в системе под номером: 10853
>Ваш вопрос:
>У вас сказано, что добровольное взятие анализа платное, и взятие анализа без медицинских показаний тоже платное. А если у нас нет показаний и нас обязывают пройти этот тест - это же уже не добровольное прохождение, подскажите пожалуйста, где в Ростове на Дону мы сможем пройти обязательное тестирование по полису? Жду ответ -- Отправлено из Mail.ru для Android четверг, 03 сентября 2020г., 12:54 +03:00 от covid@78cge.ru :> Добрый день, Ваше сообщение зарегистрировано в системе под номером: 10844> Ваш вопрос:> Здравствуйте, мы 6 дней были в отпуске, а по приезду узнали об обязательном прохождении теста на коронавирус. Мы хотим его пройти, но у нас нет средств, а в поликлинике по страховому полису нам 

In [14]:
rating = mat.dot(query.T)
rating.T[0]

array([53.79921003, 43.15390323, 29.81685085, ..., 48.51248618,
       24.3228483 , 50.47812578])

In [15]:
buf = list(zip(rating.T[0], answers_quest['join_num'].values))

for counter, val in enumerate(sorted(buf, key=lambda x: -x[0])):
    print(str(val[0]) + ": " + str(int(val[1])))
    if counter >= 20:
        break

270.4803782269762: 6
260.7573089866613: 37
251.63200801076133: 308
251.14614555252726: 308
165.773581167407: 308
162.20316093606698: 308
152.47634510890626: 308
146.77803645460517: 6
142.0738479047474: 308
139.24412919643268: 6
135.1826753143332: 37
133.82942779353738: 57
133.4118683403513: 5
125.02064154170101: 308
121.77526751281377: 308
114.55193596378707: 308
111.10297795015586: 1
110.00184803440646: 6
107.20922996864019: 308
104.26449334207283: 6
100.32660168545434: 308
